<a href="https://colab.research.google.com/github/castrokelly/Data-Science/blob/main/Abordagem_Ensemble_para_Detec%C3%A7%C3%A3o_e_Classifica%C3%A7%C3%A3o_de_Anomalias_em_Po%C3%A7os_de_Petr%C3%B3leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Ensemble para Detecção e Classificação de Anomalias em Poços de Petróleo: um Estudo Aplicado ao Dataset 3W

**Centro de Ciências Matemáticas Aplicadas à Indústria (CeMEAI)**</br>
**Instituto de Ciências Matemáticas e de Computação (ICMC)**</br>
**Universidade de São Paulo**</br>
</br>
Aluna: **Kelly Christine Alvarenga de Castro**
Área de concentração: Ciências de Dados
Orientador: **Prof. Dr. Cláudio Fabiano Motta Toledo**
</br>
---

Este Notebook apresenta o desenvolvimento de um pipeline de aprendizado de máquina para detectar e classificar anomalias em poços de petróleo utilizando o dataset 3W.

A abordagem utiliza um modelo ensemble que, em primeiro lugar, decide se o estado é anômalo (classificação binária: normal vs. anômalo) e, caso seja anômalo, classifica o tipo de evento (rótulos de 1 a 9). O treinamento é realizado com dados de um poço e a validação das métricas é efetuada com dados de outro poço.



Clonagem do repositório contendo o dataset 3W e instalação das bibliotecas necessárias:

In [94]:
!git clone https://github.com/petrobras/3W.git

fatal: destination path '3W' already exists and is not an empty directory.


In [95]:
!pip install scikeras #para a integração do Keras com o scikit-learn
!pip install PyWavelets #para a transformação wavelet

Importação das bibliotecas necessárias para manipulação dos dados, pré-processamento, construção e avaliação dos modelos:

In [105]:
# Importação das Bibliotecas Necessárias
import os
import glob
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import pywt

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Para o modelo LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline


### Funções para Carregamento e Pré-processamento dos Dados

Carregamento dos Dados

Cada arquivo Parquet é carregado e uma coluna extra “instance” (identificando o arquivo) é adicionada para posterior agregação.

In [97]:
def load_well_data(paths):
    dfs = []
    for file in paths:
        df = pd.read_parquet(file)
        df['instance'] = os.path.basename(file)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


Definindo os caminhos dos arquivos

In [98]:
# Caminhos dos arquivos para o poço 41
paths_well41 = [
    "/content/3W/dataset/9/WELL-00041_20181013160201.parquet",
    "/content/3W/dataset/9/WELL-00041_20190814154301.parquet",
    "/content/3W/dataset/9/WELL-00041_20190817173916.parquet",
    "/content/3W/dataset/9/WELL-00041_20190819163753.parquet",
    "/content/3W/dataset/9/WELL-00041_20190822143519.parquet",
    "/content/3W/dataset/9/WELL-00041_20190906051807.parquet",
    "/content/3W/dataset/9/WELL-00041_20190917161232.parquet",
    "/content/3W/dataset/9/WELL-00041_20190924202109.parquet",
    "/content/3W/dataset/9/WELL-00041_20190925111412.parquet"
]

# Caminhos dos arquivos para o poço 42
paths_well42 = [
    "/content/3W/dataset/9/WELL-00042_20141217142745.parquet",
    "/content/3W/dataset/9/WELL-00042_20141218004109.parquet",
    "/content/3W/dataset/9/WELL-00042_20141218051903.parquet",
    "/content/3W/dataset/9/WELL-00042_20141221190219.parquet",
    "/content/3W/dataset/9/WELL-00042_20141222024535.parquet"
]

Carregando os dados dos poços:

In [99]:
df_well41 = load_well_data(paths_well41)
df_well42 = load_well_data(paths_well42)

## Definição da Flag de Anomalia

Cria-se a coluna "is_anomaly" a partir da coluna "class". Segundo as regras:

Se "class" for 0, a operação é normal (is_anomaly = 0);

Caso contrário (por exemplo, 9 ou 109), is_anomaly = 1.

In [100]:
df_well41['is_anomaly'] = df_well41['class'].apply(lambda x: 0 if x == 0 else 1)
df_well42['is_anomaly'] = df_well42['class'].apply(lambda x: 0 if x == 0 else 1)

## Agregação dos Dados e Extração de Features:

Para cada instância (arquivo), calcula-se as estatísticas (média, std, mínimo e máximo) das colunas de sensores. Na agregação da coluna `"class"`, utiliza-se o valor máximo, de forma que:

* Se todos os valores forem 0, a instância é normal;
* Se houver pelo menos um 9, o máximo será 9;
* Se houver pelo menos um 109, o máximo será 109.

Essa abordagem também é aplicada para "is_anomaly" (usando o .max()).

In [101]:
def aggregate_features(df, sensor_cols):
    agg_funcs = ['mean', 'std', 'min', 'max']
    agg_df = df.groupby('instance')[sensor_cols].agg(agg_funcs)
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]

    # Agregação das colunas "class" e "is_anomaly" usando .max()
    classes = df.groupby('instance')['class'].max()
    is_anomaly = df.groupby('instance')['is_anomaly'].max()

    agg_df['class'] = classes
    agg_df['is_anomaly'] = is_anomaly
    return agg_df

In [108]:
# Selecionar as colunas de sensores: todas as colunas numéricas, exceto as de identificação
cols_exclude = ['timestamp', 'instance', 'class', 'is_anomaly']
sensor_cols = [col for col in df_well41.columns if col not in cols_exclude and pd.api.types.is_numeric_dtype(df_well41[col])]

agg_well41 = aggregate_features(df_well41, sensor_cols)
agg_well42 = aggregate_features(df_well42, sensor_cols)

## Pré-Processamento: Normalização e Redução de Dimensionalidade

Agora, vamos separar as features e os rótulos (usando o target binário `is_anomaly`), em seguida:

* Divisão do conjunto do poço 41 em treino e validação;
* Normalização com `StandardScaler`;
* Redução de dimensionalidade com PCA (mantendo 95% da variância).

In [109]:
agg_well41_clean = agg_well41.dropna(axis=1)
agg_well42_clean = agg_well42.dropna(axis=1)

In [111]:
# Como já temos a coluna "class" agregada, removemos apenas "class" e "is_anomaly" das features.
features_well41 = agg_well41_clean.drop(['class','is_anomaly'], axis=1)
features_well42 = agg_well42_clean.drop(['class','is_anomaly'], axis=1)
target_well41 = agg_well41_clean['is_anomaly']
target_well42 = agg_well42_clean['is_anomaly']

selector = VarianceThreshold(threshold=0)
features_well41_sel = selector.fit_transform(features_well41)
features_well42_sel = selector.transform(features_well42)
features_selected = features_well41.columns[selector.get_support()]

X_well41 = pd.DataFrame(features_well41_sel, columns=features_selected, index=features_well41.index)
X_well42 = pd.DataFrame(features_well42_sel, columns=features_selected, index=features_well42.index)


In [117]:
# Divisão dos dados do poço 41 em treino (70%) e validação (30%)
X_train, X_val, y_train, y_val = train_test_split(
    X_well41, target_well41, test_size=0.3, random_state=42, stratify=target_well41
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_train_full_scaled = scaler.transform(X_well41)
X_test_deploy_scaled = scaler.transform(X_well42)

In [113]:
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca   = pca.transform(X_val_scaled)
X_train_full_pca = pca.transform(X_train_full_scaled)
X_test_deploy_pca = pca.transform(X_test_deploy_scaled)

n_features = X_train_pca.shape[1]
print("Número de features após PCA:", n_features)

Número de features após PCA: 4


## Preparação para o Modelo LSTM

Adiciona uma dimensão temporal (timestep único) para que os dados possam ser processados pelo LSTM.

In [114]:
X_train_pca_lstm = X_train_pca.reshape(-1, 1, n_features)
X_val_pca_lstm   = X_val_pca.reshape(-1, 1, n_features)
X_train_full_pca_lstm = X_train_full_pca.reshape(-1, 1, n_features)
X_test_deploy_pca_lstm = X_test_deploy_pca.reshape(-1, 1, n_features)

## Definição dos Modelos e Criação do Ensemble

São definidos três modelos:

* **Random Forest** e **Gradient Boosting** para capturar relações não lineares.

* **LSTM** para modelar dependências temporais, encapsulado via scikeras.

Os modelos são integrados em um `ensemble via stacking` (meta-classificador: regressão logística).


In [115]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm_clf = KerasClassifier(model=create_lstm_model,
                             model__input_shape=(1, n_features),
                             epochs=10, batch_size=16, verbose=0)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)

estimators = [
    ('rf', rf_clf),
    ('gb', gb_clf),
    ('lstm', lstm_clf)
]

stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)


In [119]:
from collections import Counter
print("Class distribution in y_train:", Counter(y_train))

Class distribution in y_train: Counter({1: 6})


## Treinamento e Avaliação Interna (Poço 41)

Treina o ensemble com os dados de treino do poço 41 e avalia no conjunto de validação.

São calculadas as métricas: acurácia, F1-Score, ROC AUC, matriz de confusão, especificidade e FAR.

In [118]:
stack.fit(X_train_pca, y_train)
y_val_pred = stack.predict(X_val_pca)
f1 = f1_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, stack.predict_proba(X_val_pca)[:,1])
cm = confusion_matrix(y_val, y_val_pred)

print("### Métricas de Validação Interna (Poço 41)")
print("Acurácia:", accuracy)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("Matriz de Confusão:\n", cm)

tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
far = fp / (tn + fp) if (tn + fp) != 0 else 0
print("Especificidade (SPE):", specificity)
print("Taxa de Falsos Alarmes (FAR):", far)

ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.